In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
df = (
    spark
    .read
    .format('parquet')
    .load('abfss://bronze@databricksstorageete.dfs.core.windows.net/products')
)

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView('products')

### Functions

In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.less_10_perc_func(price DOUBLE)
RETURNS DOUBLE 
LANGUAGE SQL
RETURN price * 0.90


In [0]:
%sql
CREATE OR REPLACE FUNCTION databricks_cata.bronze.take_10_perc_func(price DOUBLE)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
  return price * 0.1
$$

In [0]:
%sql
SELECT product_id, price, 
  ROUND(databricks_cata.bronze.take_10_perc_func(price), 2) AS discount,
  ROUND(databricks_cata.bronze.less_10_perc_func(price), 2) AS discounted_price
FROM products;

In [0]:
transformed_df = (
  df
  .withColumn('update_date', F.current_timestamp())
  .withColumn('discount', F.expr('ROUND(databricks_cata.bronze.take_10_perc_func(price), 2)'))
  .withColumn('discounted_price', F.expr('ROUND(databricks_cata.bronze.less_10_perc_func(price), 2)'))
)

In [0]:
transformed_df.display()

In [0]:
(
    transformed_df
    .write
    .format('delta')
    .option('mergeSchema', 'true')
    .mode('overwrite')
    .save('abfss://silver@databricksstorageete.dfs.core.windows.net/products')
)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.products
USING DELTA
LOCATION 'abfss://silver@databricksstorageete.dfs.core.windows.net/products';

In [0]:
%sql
SELECT * FROM databricks_cata.silver.products;